# Phase 1 & 2 - Data Engineering & NLP : Prétraitement et Analyse de Sentiment

Ce notebook démontre :
1. Le nettoyage des tweets collectés
2. L'analyse de sentiment avec VADER et TextBlob
3. La classification des sentiments
4. L'identification des tweets les plus négatifs



In [ ]:
# Importation des bibliothèques
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from src.preprocess_tesla import TeslaTextPreprocessor
from src.analyze_tesla_sentiment import TeslaSentimentAnalyzer

print("✅ Bibliothèques importées")



## Étape 1 : Chargement des données brutes


In [ ]:
# Charger les données brutes
df_raw = pd.read_csv("../data/tesla_tweets_raw.csv")
print(f"📊 {len(df_raw)} tweets chargés")

# Afficher un exemple de tweet brut
print("\n📝 Exemple de tweet brut :")
print(df_raw['text'].iloc[0])



## Étape 2 : Prétraitement des tweets

Le prétraitement effectue les opérations suivantes :
- Suppression des liens HTTP/HTTPS
- Suppression des mentions @user
- Suppression de la ponctuation et des chiffres
- Conversion en minuscules
- Suppression des stopwords
- Extraction de features spécifiques à Tesla


In [ ]:
# Initialiser le preprocessor
preprocessor = TeslaTextPreprocessor(language='english', lemmatize=False)

# Nettoyer les tweets
df_cleaned = preprocessor.preprocess_dataframe(df_raw)

# Comparer avant/après
print("\n🔍 Comparaison avant/après nettoyage :\n")
comparison_df = pd.DataFrame({
    'Avant': df_raw['text'].head(3).values,
    'Après': df_cleaned['text_cleaned'].head(3).values
})
comparison_df



In [ ]:
# Sauvegarder les données nettoyées
df_cleaned.to_csv("../data/tesla_tweets_cleaned.csv", index=False)
print("💾 Données nettoyées sauvegardées")

# Statistiques sur les features Tesla
print("\n📈 Features Tesla extraites :")
print(f"   Tweets mentionnant un modèle : {df_cleaned['mentions_model'].sum()}")
print(f"   Tweets mentionnant Elon Musk : {df_cleaned['mentions_elon'].sum()}")
print(f"   Tweets mentionnant la compagnie : {df_cleaned['mentions_company'].sum()}")



## Étape 3 : Analyse de sentiment

Nous utilisons deux méthodes :
- **VADER** : Spécialement conçu pour les réseaux sociaux
- **TextBlob** : Méthode classique basée sur des règles

**Classification :**
- Positif : polarité > 0.1
- Négatif : polarité < -0.1
- Neutre : -0.1 ≤ polarité ≤ 0.1


In [ ]:
# Initialiser l'analyseur
analyzer = TeslaSentimentAnalyzer()

# Analyser le sentiment
df_analyzed = analyzer.analyze_dataframe(df_cleaned)

# Afficher un aperçu
print("\n📊 Aperçu des résultats d'analyse :")
df_analyzed[['text_cleaned', 'polarity', 'sentiment', 'vader_compound', 'textblob_polarity']].head(10)



In [ ]:
# Statistiques globales
stats = analyzer.get_statistics(df_analyzed)

print("📈 Statistiques de sentiment :")
print(f"   Total tweets : {stats['total_tweets']}")
print(f"   Positifs : {stats['positive_count']} ({stats['positive_percent']:.1f}%)")
print(f"   Négatifs : {stats['negative_count']} ({stats['negative_percent']:.1f}%)")
print(f"   Neutres : {stats['neutral_count']} ({stats['neutral_percent']:.1f}%)")
print(f"   Polarité moyenne : {stats['mean_polarity']:.3f} ± {stats['std_polarity']:.3f}")



## Étape 4 : Visualisation de la distribution des sentiments


In [ ]:
# Graphique en barres de la distribution
sentiment_counts = df_analyzed['sentiment'].value_counts()

plt.figure(figsize=(10, 6))
colors = {'positive': '#2E7D32', 'negative': '#C62828', 'neutral': '#757575'}
bars = plt.bar(sentiment_counts.index.str.title(), sentiment_counts.values, 
               color=[colors.get(s.lower(), '#757575') for s in sentiment_counts.index])

plt.title('Distribution des Sentiments', fontsize=16, fontweight='bold')
plt.xlabel('Sentiment', fontsize=12)
plt.ylabel('Nombre de tweets', fontsize=12)

# Ajouter les valeurs sur les barres
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height)}\n({height/len(df_analyzed)*100:.1f}%)',
             ha='center', va='bottom')

plt.tight_layout()
plt.show()



In [ ]:
# Histogramme de la distribution des polarités
plt.figure(figsize=(12, 6))
plt.hist(df_analyzed['polarity'], bins=50, edgecolor='black', alpha=0.7, color='#E31937')
plt.axvline(x=0.1, color='green', linestyle='--', label='Seuil positif (>0.1)')
plt.axvline(x=-0.1, color='red', linestyle='--', label='Seuil négatif (<-0.1)')
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)
plt.title('Distribution des Scores de Polarité', fontsize=16, fontweight='bold')
plt.xlabel('Polarité', fontsize=12)
plt.ylabel('Fréquence', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()



## Étape 5 : Comparaison VADER vs TextBlob


In [ ]:
# Comparaison des classifications
comparison = pd.crosstab(df_analyzed['sentiment_vader'], df_analyzed['sentiment_textblob'], 
                         margins=True, margins_name="Total")

print("📊 Matrice de confusion VADER vs TextBlob :")
print(comparison)

# Visualisation
plt.figure(figsize=(8, 6))
sns.heatmap(comparison.iloc[:-1, :-1], annot=True, fmt='d', cmap='YlOrRd', 
            cbar_kws={'label': 'Nombre de tweets'})
plt.title('Comparaison des Classifications VADER vs TextBlob', fontsize=14, fontweight='bold')
plt.xlabel('TextBlob', fontsize=12)
plt.ylabel('VADER', fontsize=12)
plt.tight_layout()
plt.show()



## Étape 6 : Identification des tweets les plus négatifs


In [ ]:
# Obtenir les 5 tweets les plus négatifs
top_negative = analyzer.get_top_negative_tweets(df_analyzed, n=5)

print("🔴 Top 5 tweets les plus négatifs :\n")
for idx, (_, row) in enumerate(top_negative.iterrows(), 1):
    print(f"{'='*80}")
    print(f"Tweet #{idx} - Polarité: {row['polarity']:.3f}")
    print(f"Score négatif VADER: {row['vader_neg']:.3f}")
    print(f"Likes: {row.get('likes', 0)} | RT: {row.get('retweets', 0)}")
    print(f"\nTexte original :\n{row['text']}")
    print(f"\nTexte nettoyé :\n{row['text_cleaned']}")
    
    # Détecter le sarcasme
    sarcasm_indicators = analyzer.detect_sarcasm_indicators(row['text'])
    if sarcasm_indicators:
        print(f"\n⚠️  Indicateurs de sarcasme : {', '.join(sarcasm_indicators)}")
    print()



In [ ]:
# Sauvegarder les résultats finaux
df_analyzed.to_csv("../data/tesla_sentiment_results.csv", index=False)
print("💾 Résultats d'analyse sauvegardés dans ../data/tesla_sentiment_results.csv")



## Conclusion

✅ **Prétraitement et analyse terminés !**

- ✅ Tweets nettoyés et prétraités
- ✅ Analyse de sentiment effectuée avec VADER et TextBlob
- ✅ Classification des sentiments (Positif/Négatif/Neutre)
- ✅ Identification des tweets les plus négatifs
- ✅ Données sauvegardées pour le dashboard

**Prochaine étape** : Utiliser le dashboard Streamlit (`python src/tesla_dashboard.py`) ou le notebook `3_visualize_dashboard.ipynb` pour visualiser les résultats.

